In [1]:
import os, time, json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from pathlib import Path


2026-02-06 13:45:38.763876: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-06 13:45:38.925766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-06 13:45:39.000620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-06 13:45:39.000903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-06 13:45:39.134660: I tensorflow/core/platform/cpu_feature_gua

In [2]:
# CONFIGURATION
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

EPOCHS_HEAD = 10
EPOCHS_FINE = 15

LR_HEAD = 1e-3
LR_FINE = 1e-4

DATASET_DIR = Path("/home/admin1/PROJETS_KEYCE/cc_transfert/dataset")
TRAIN_DIR = DATASET_DIR / "train"
VAL_DIR   = DATASET_DIR / "val"

assert TRAIN_DIR.exists() and VAL_DIR.exists()
print("✅ Dataset OK")


✅ Dataset OK


**Data Augmentation**

In [3]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)
val_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [4]:
train_ds = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=SEED
)
val_ds = val_gen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)
NUM_CLASSES = train_ds.num_classes
CLASS_NAMES = list(train_ds.class_indices.keys())

print("Classes:", CLASS_NAMES)
print("NUM_CLASSES:", NUM_CLASSES)

Found 1840 images belonging to 10 classes.
Found 466 images belonging to 10 classes.
Classes: ['crease', 'crescent_gap', 'inclusion', 'oil_spot', 'punching_hole', 'rolled_pit', 'silk_spot', 'waist_folding', 'water_spot', 'welding_line']
NUM_CLASSES: 10


**Modèle EfficientNetB0**

In [5]:
def build_efficientnet(num_classes, lr):
    base_model = EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=(*IMG_SIZE, 3)
    )
    base_model.trainable = False

    inputs = keras.Input(shape=(*IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model, base_model


In [6]:
model, base_model = build_efficientnet(NUM_CLASSES, LR_HEAD)
model.summary()


2026-02-06 13:45:42.034772: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-06 13:45:42.658489: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385,197 (16.73 MB)

 Trainable params: 333,066 (1.27 MB)

 Non-trainable params: 4,052,131 (15.46 MB)

**Entraînement HEAD ONLY**

In [7]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,
        restore_best_weights=True
    )
]
history_head = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEAD,
    callbacks=callbacks
)

Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 47s 709ms/step - accuracy: 0.5967 - loss: 1.3543 - val_accuracy: 0.6845 - val_loss: 1.0770
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 565ms/step - accuracy: 0.7310 - loss: 0.8592 - val_accuracy: 0.7425 - val_loss: 0.8696
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 35s 600ms/step - accuracy: 0.7429 - loss: 0.7787 - val_accuracy: 0.7639 - val_loss: 0.7785
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 35s 596ms/step - accuracy: 0.7652 - loss: 0.7179 - val_accuracy: 0.7961 - val_loss: 0.6261
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 530ms/step - accuracy: 0.7975 - loss: 0.6052

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 341, in dispatch_control
    await self.process_control(msg)
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 347, in process_control
    idents, msg = self.session.feed_identities(msg, copy=False)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/jupyter_client/session.py", line 998, in feed_identities
    raise ValueError(msg)
ValueError: DELIM not in msg_list
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/admin1/PRO

58/58 ━━━━━━━━━━━━━━━━━━━━ 36s 627ms/step - accuracy: 0.7815 - loss: 0.6493 - val_accuracy: 0.7940 - val_loss: 0.6157
Epoch 6/10
23/58 ━━━━━━━━━━━━━━━━━━━━ 18s 526ms/step - accuracy: 0.7642 - loss: 0.6943

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 341, in dispatch_control
    await self.process_control(msg)
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 347, in process_control
    idents, msg = self.session.feed_identities(msg, copy=False)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/jupyter_client/session.py", line 998, in feed_identities
    raise ValueError(msg)
ValueError: DELIM not in msg_list
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/admin1/PRO

58/58 ━━━━━━━━━━━━━━━━━━━━ 35s 610ms/step - accuracy: 0.7891 - loss: 0.6032 - val_accuracy: 0.8155 - val_loss: 0.5386
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 35s 599ms/step - accuracy: 0.7978 - loss: 0.5914 - val_accuracy: 0.8004 - val_loss: 0.5854
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 36s 611ms/step - accuracy: 0.8082 - loss: 0.5646 - val_accuracy: 0.8026 - val_loss: 0.5931
Epoch 9/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 35s 608ms/step - accuracy: 0.8168 - loss: 0.5494 - val_accuracy: 0.8412 - val_loss: 0.5552
Epoch 10/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 36s 618ms/step - accuracy: 0.8147 - loss: 0.5383 - val_accuracy: 0.8155 - val_loss: 0.5870


**Fine-Tuning partiel (30 dernières couches)**

In [8]:
# Dégel partiel contrôlé
total_layers = len(base_model.layers)
FINE_TUNE_AT = total_layers - 30

for layer in base_model.layers[:FINE_TUNE_AT]:
    layer.trainable = False

for layer in base_model.layers[FINE_TUNE_AT:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

print("Couches entraînables:",
      sum(l.trainable for l in base_model.layers))

Couches entraînables: 23


**Entraînement Fine-Tuning**

In [9]:
callbacks_fine = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=4,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=2,
        min_lr=1e-6
    )
]
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINE,
    callbacks=callbacks_fine
)

Epoch 1/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 35s 597ms/step - accuracy: 0.7978 - loss: 0.5774 - val_accuracy: 0.8197 - val_loss: 0.5972 - learning_rate: 0.0010
Epoch 2/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 33s 576ms/step - accuracy: 0.8299 - loss: 0.5219 - val_accuracy: 0.8240 - val_loss: 0.5590 - learning_rate: 0.0010
Epoch 3/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 34s 584ms/step - accuracy: 0.8120 - loss: 0.5542 - val_accuracy: 0.8069 - val_loss: 0.5742 - learning_rate: 0.0010
Epoch 4/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 34s 582ms/step - accuracy: 0.8375 - loss: 0.5085 - val_accuracy: 0.8412 - val_loss: 0.5351 - learning_rate: 0.0010
Epoch 5/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 35s 596ms/step - accuracy: 0.8234 - loss: 0.5122 - val_accuracy: 0.8326 - val_loss: 0.5370 - learning_rate: 0.0010
Epoch 6/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 34s 580ms/step - accuracy: 0.8207 - loss: 0.5113 - val_accuracy: 0.8155 - val_loss: 0.6269 - learning_rate: 0.0010
Epoch 7/15
58/58 ━━━━━━━━━━━━━━━━━━━━ 34s 592ms/step - accuracy: 0.8484 - loss: 0.

**Évaluation finale**

In [10]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"✅ EfficientNetB0 – Val accuracy finale : {val_acc:.4f}")


15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 316ms/step - accuracy: 0.8433 - loss: 0.5183
✅ EfficientNetB0 – Val accuracy finale : 0.8433


**Sauvegarde modèle + métadonnées**

In [ ]:
MODEL_PATH = "efficientnetb0_gc10_finetuned.h5"
model.save(MODEL_PATH)
metadata = {
    "model": "EfficientNetB0",
    "img_size": IMG_SIZE,
    "num_classes": NUM_CLASSES,
    "val_accuracy": float(val_acc)
}
with open("efficientnetb0_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print("💾 Modèle et métadonnées sauvegardés")

💾 Modèle et métadonnées sauvegardés


ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 341, in dispatch_control
    await self.process_control(msg)
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 347, in process_control
    idents, msg = self.session.feed_identities(msg, copy=False)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/admin1/PROJETS_KEYCE/cc_cnn/venv_cnn/lib/python3.12/site-packages/jupyter_client/session.py", line 998, in feed_identities
    raise ValueError(msg)
ValueError: DELIM not in msg_list
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/admin1/PRO